In [1]:
from tqdm import tqdm
from time import sleep
import requests
import re
import json
import pandas as pd
import csv
from selenium.webdriver import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
def google_reviews(place_list,gu,count=10):
    
    #크롬 드라이버 실행
    chrome_options = Options()
    chrome_options.add_experimental_option("detach",True)
    chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

    service = Service(executable_path=ChromeDriverManager().install())
    driver=webdriver.Chrome(service=service, options=chrome_options)
    driver.get('https://www.google.com/maps/?hl=en-US')
    driver.implicitly_wait(10)
    
    #리스트 형식으로 저장된 장소 하나씩 검색
    for place in place_list:
        count=count
        driver.get('https://www.google.com/maps/?hl=en-US') #구글맵 영어 창 열기
        time.sleep(3)
        
        #검색창에 검색
        search=driver.find_element(By.CSS_SELECTOR,'#searchboxinput') #검색창 열기
        search.send_keys(place+' '+gu+'\n') #주소 + 장소이름 검색
        
        #리뷰보기
        review_btn=wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]')))
        review_btn.click()
        time.sleep(1)
        #views=int(''.join(review_btn.text[4:-7].split(',')))-1
        #print(views)
        
        #최신순정렬
        sort_btn = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button/span').click()
        time.sleep(1)
        newest_btn = driver.find_element(By.XPATH,'//*[@id="action-menu"]/ul/li[2]').click()
        #action-menu
        
       #div태그 스크롤
        for i in range(2000):
            try:
                scroll = driver.find_element(By.CSS_SELECTOR,'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
                scroll.send_keys(Keys.PAGE_DOWN)
                if i/10==0:
                    time.sleep(1)

            except Exception as e:
                print(e)
                break
        
        
        #스크래핑
        reviews = driver.find_elements(By.CSS_SELECTOR,'.jftiEf')
        name=[]
        star=[]
        content=[]
        date=[]
        for review in reviews:
            try:
                more_btn = review.find_element(By.CSS_SELECTOR,'.w8nwRe.kyuRq').click()
                print("y",end="")
                WebDriverWait(review,60).until(EC.invisibility_of_element_located((By.CSS_SELECTOR,".w8nwRe.kyuRq")))
                name.append(review.find_element(By.CSS_SELECTOR,'.d4r55').text)
                star.append(int(float(review.find_element(By.CSS_SELECTOR,'.kvMYJc').get_attribute("aria-label")[1])))
                content.append(review.find_element(By.CSS_SELECTOR,'.wiI7pd').text.replace("\n"," "))
                date.append(review.find_element(By.CSS_SELECTOR,'.rsqaWe').text)
            except:
                print("n",end="")
                name.append(review.find_element(By.CSS_SELECTOR,'.d4r55').text)
                star.append(int(float(review.find_element(By.CSS_SELECTOR,'.kvMYJc').get_attribute("aria-label")[1])))
                content.append(review.find_element(By.CSS_SELECTOR,'.wiI7pd').text.replace("\n"," "))
                date.append(review.find_element(By.CSS_SELECTOR,'.rsqaWe').text)
        
        emoji = re.compile("["
                    u"\U0001F600-\U0001F64F"
                    u"\U0001F300-\U0001F5FF"
                    u"\U0001F680-\U0001F6FF"
                    u"\U0001F1E0-\U0001F1FF"
                       "]+", flags=re.UNICODE)
        
        #이모티콘,특수문자, 한글 제거
        content_noemj=[]
        name_noko=[]
        
        korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')

        for i in name :
            i = re.sub(korean, ' KoReAn ', i)
            i = i.encode('utf-8').decode('ascii','ignore')
            name_noko.append(i)
            
        for i in content :
            i = emoji.sub(r'',i)
            i = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\₩[\]\<\>`\'…《\》’]', '', i)
            i = re.sub(korean, ' KoReAn ', i) 
            i = i.encode('utf-8').decode('ascii','ignore')
            content_noemj.append(i)
            
        print(len(content),len(name))
        all=pd.DataFrame([name_noko,star,content_noemj,date],index=['NAME','STAR','CONTENT','DATE']).transpose()
        print(content_noemj)
        file_name = '{}review.csv'.format(place+' ')
        all.to_csv(file_name)
    
                


In [3]:
place_list=['Deoksugung','War Memorial Korea','National Museum of Korea']
google_reviews(place_list,'Seoul',5)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.13M/6.13M [00:00<00:00, 9.62MB/s]


nynnynynnynnynynnnynynynynnnynnynynnynynnnynnynynynnnynnynnynnnnnynnynynynnnynynnynnynynynnynynnynnynnnnynynnynnnynynynnnynnnynynynynnnnynynynnnnynnynnynynynnynnnynnnnynnynnnnnynnnynnnynnnnnnynynnnynnnnnnynynynnnynynynnynnynnnnnnnynynnynynnynnynynnynnnnnynynnynynnynnnnynynnynnynnnynnynynnynynynynnynynnnnnnnnynnnnynnynnnnnnynynnnnynnnnnnnynynynynynnnynnnnnnnnnnnynnnynynynnnynnynnnnnynnnynynynnnynynnynnnnnnnnnnnnnynnynynnynynnnynnynynnnnnnynynnnnnynnnnnnnynnynnnynnynnnynnnnynnynynynynnynnynnnnnnynnynnynnynynynnynynnynnnnnnnnnnynnynynynnynnnnynynnnnynynnynnnynynnynnynnynynnnynnnynnynnnnynynnynynnynnnnnnynnnnynnynnynynnnynnynnnynnynynynnnnnnynnnnnnnnnynynnynnynynnnnynynynynynynynnnynynynnynnnnnnnnnnnnnnnnnynnnynnnnnynynnnnnynnynnnynnnnnynnnnnnnnynynnnnnnnnnynnynnnnnynnnnnynynynnnnnnnnnnnnynynynnnynynynynnnynnnnnnynnnynynnynnnynnynnnnnynnnynynnynynnnnnnnnynnnnnnnnynnnnnnnnnnnnnnnnnnnnnnnnnynnnnynnnnnnnnnnnynnnnnnnynnnnnynnynnnnnnnynnnnnnnnnynnnnynnynnnynnynnnnnnnynnnnnnnnnnnnnynynnnnnnnnynn